# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis = 1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [5]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 

reader = Reader()
surp_df = Dataset.load_from_df(new_df, reader)

Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [7]:
dataset = surp_df.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [8]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [9]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run

params = {'n_factors': [20, 50, 100],
         'reg_all': [0.02, 0.05, 0.1]}

g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
g_s_svd.fit(surp_df)

In [11]:
# print out optimal parameters for SVD after GridSearch
g_s_svd.best_params

{'rmse': {'n_factors': 100, 'reg_all': 0.05},
 'mae': {'n_factors': 100, 'reg_all': 0.05}}

In [12]:
# cross validating with KNNBasic
basic = KNNBasic(sim_options={'name': 'pearson', 'user_based': True})
cv_knn_basic = cross_validate(basic, surp_df, n_jobs=-1)

In [14]:
# print out the average RMSE score for the test set
for i in cv_knn_basic.items():
    print(i)
print('-----------------------')
print(np.mean(cv_knn_basic['test_rmse']))

('test_rmse', array([0.96995658, 0.97524005, 0.96166749, 0.97641893, 0.9784485 ]))
('test_mae', array([0.74712738, 0.75319293, 0.74253057, 0.75450992, 0.75619196]))
('fit_time', (1.694328784942627, 1.948981761932373, 2.1542389392852783, 2.1689682006835938, 2.3129067420959473))
('test_time', (5.9550018310546875, 5.717710733413696, 5.68215799331665, 5.581748008728027, 5.462432861328125))
-----------------------
0.9723463121084077


In [15]:
# cross validating with KNNBaseline
base = KNNBaseline(sim_options = {'name' : 'pearson', 'user_based' : True})
cv_knn_baseline = cross_validate(base, surp_df)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.


In [16]:
# print out the average score for the test set
for i in cv_knn_baseline.items():
    print(i)

np.mean(cv_knn_baseline['test_rmse'])

('test_rmse', array([0.87709935, 0.88249793, 0.87230232, 0.87489212, 0.87419866]))
('test_mae', array([0.67016201, 0.67343963, 0.66765173, 0.66695587, 0.66955627]))
('fit_time', (1.7256171703338623, 1.8387634754180908, 2.836798906326294, 1.9613566398620605, 2.082296133041382))
('test_time', (4.428506374359131, 4.470096588134766, 5.049233436584473, 4.3809685707092285, 3.953131675720215))


0.8761980758519382

Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [17]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [18]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [19]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [20]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=2.9624911286574735, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [21]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list
        

In [22]:
# try out the new function here!
user_rating = movie_rater(df_movies, 4, 'Comedy')

      movieId                                      title          genres
6415    51082  Tyler Perry's Daddy's Little Girls (2007)  Comedy|Romance


      movieId               title        genres
6943    65230  Marley & Me (2008)  Comedy|Drama
      movieId          title  genres
7585    86293  Arthur (2011)  Comedy
      movieId            title          genres
8846   132488  Lovesick (2014)  Comedy|Romance


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [23]:
user_rating

[{'userId': 1000, 'movieId': 51082, 'rating': '3'},
 {'userId': 1000, 'movieId': 65230, 'rating': '6'},
 {'userId': 1000, 'movieId': 86293, 'rating': '4'},
 {'userId': 1000, 'movieId': 132488, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [24]:
## add the new ratings to the original ratings DataFrame
user_ratings = pd.DataFrame(user_rating)
new_ratings_df = pd.concat([new_df, user_ratings], axis=0)
new_data = Dataset.load_from_df(new_ratings_df,reader)

In [25]:
# train a model using the new combined DataFrame
# train a model using the new combined DataFrame
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [26]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
list_of_movies = []
for m_id in new_df['movieId'].unique():
    list_of_movies.append((m_id,svd_.predict(1000,m_id)[3]))

In [29]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse = True)
ranked_movies

[(1204, 4.6077746000985425),
 (318, 4.574404822916052),
 (720, 4.556009814814273),
 (50, 4.519889788685533),
 (750, 4.515096646091683),
 (1276, 4.51156821935712),
 (1197, 4.510089780212765),
 (1104, 4.504515150619231),
 (741, 4.502089348031),
 (356, 4.494521218918972),
 (3030, 4.49322621461126),
 (898, 4.493108841536963),
 (260, 4.486272261684163),
 (912, 4.478339045592146),
 (904, 4.4687612111499835),
 (908, 4.462953632264852),
 (1233, 4.461922541159277),
 (2571, 4.457168731071763),
 (1196, 4.44989393195419),
 (593, 4.445455882887271),
 (3451, 4.441585330117943),
 (1248, 4.434069907398245),
 (2329, 4.428971996530789),
 (5618, 4.425453127513692),
 (1203, 4.4217993129701885),
 (1178, 4.418887962116812),
 (2019, 4.417586820678357),
 (1089, 4.415783020274052),
 (2959, 4.415257949815053),
 (858, 4.414129715912158),
 (1262, 4.414027497722092),
 (527, 4.413864004980281),
 (105504, 4.413096801953143),
 (1242, 4.405386169700501),
 (899, 4.4028464251653965),
 (1208, 4.401429554605829),
 (1267, 

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [30]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,df_movies,5)

Recommendation #  1 :  906    Lawrence of Arabia (1962)
Name: title, dtype: object 

Recommendation #  2 :  277    Shawshank Redemption, The (1994)
Name: title, dtype: object 

Recommendation #  3 :  585    Wallace & Gromit: The Best of Aardman Animatio...
Name: title, dtype: object 

Recommendation #  4 :  46    Usual Suspects, The (1995)
Name: title, dtype: object 

Recommendation #  5 :  602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object 



## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.